In [ ]:
using Pkg,Statistics,Random,Printf,GZip,Knet,Plots,LinearAlgebra,Distributions

In [ ]:
# GENERATING THE NOISY GAUSSIAN (TRAINING DATA)

In [ ]:
Random.seed!(4);
Range=3.0; # range of the x values for the target Gaussian function
Incr = 0.03; # determines the number of samples from which we'll learn
Noise_std=0.1; # add noise on the Gaussian

In [ ]:
# generate the data from which we'll learn the Gaussian function
# obligatory arguments listed before ";" while optional arguments come after ";".
function gen_noisy_gaussian(;range=1.0,noise=0.1)
    x = collect(-Range:Incr:Range)
    y = exp.(-x.^2) + randn(length(x))*noise; # additive gaussian noise
    return (x,y)
end
# output is two vectors x,y

In [ ]:
(x_train,y_train) = gen_noisy_gaussian(range=Range,noise=Noise_std);

In [ ]:
Ntrain =length(x_train) # number of training data points

In [ ]:
plot(x_train,[y_train,exp.(-x_train.^2)])

In [ ]:
# CONSTRUCT THE NETWORK AND THE LOSS FUNCTION

In [ ]:
HiddenSize = 2; # number of neurons in the hidden layer
Batchsize = 10;
RegWeight=0.001; # lambda for L2 regularization
InitNorm = 0.5; # initial weight norm

In [ ]:
# The deep learning package requires a certain array structure for the weights
# but it is easier for the later analysis to dump them all into a single column vector
function flat(w) # make a single vector out of all weights
    return vcat(w[1],w[2],w[3],w[4])
end

In [ ]:
# reconstruct the weight array from the flat weight vector
function unflat(wf)
    return [wf[1:HiddenSize],wf[HiddenSize+1:2*HiddenSize],wf[2*HiddenSize+1:3*HiddenSize],wf[end]]
end

In [ ]:
# one layer network performing: tanh.(w[hidden,input] * x[input,batchsize] .+ b[hidden,1])
# The dot "." is for "broadcasting": performing the operation pointwise on each input element

In [ ]:
# Change this seed to try different initial weigths w/o changing the training data
Random.seed!(1);

In [ ]:
# initialize weights: w = [w1,w2,w3,w4] -> output = w3*tanh.(w1*x .+ w2) .+ w4
w = [rand(HiddenSize),rand(HiddenSize),rand(HiddenSize),rand()];
w = InitNorm*w/norm(flat(w)); # rescale w so that the norm is InitNorm
Nweights = length(flat(w));

In [ ]:
# dimensions:
# x: (input x batchsize) - input
# w[1]: (hidden x input) - (input->hidden) weights
# w[2]: (hidden x 1) - (input->hidden) biases
# w[3]: (hidden x output) - (hidden-> output) weights
# w[4]: (output x 1) - (hidden->output) bias

# both x and y are ordered in columns per training data point
function loss(w,x,y)
    guesses =  w[3]'*tanh.(w[1]*x' .+ w[2]) .+ w[4];
    return mean(abs2,y'-guesses) + RegWeight*sum(norm(w[i])^2  for i=1:4)
end


In [ ]:
# construct the gradient-calculating function. grad() is a "functional" whose input and output
# is a function. Note that grad() requires loss to be a scalar function
lossgradient = grad(loss)

In [ ]:
# calculate gradient at the initial w
# dw has dimensions of w: each weight w_i is replaced with the gradient wrt w_i
dw = lossgradient(w,x_train,y_train)

In [ ]:
# THE TRAINING FUNCTION THAT PRESENTS THE TRAINING SET IN RANDOM BATCHES (WITH NO REPLACEMENT)
# For random batches WITH replacement, move the line "perm = .." inside the for loop

function mytrain!(w;lr=0.1)
    Nbatch = floor(Int,Ntrain/Batchsize); # few training samples will be left out if Ntrain/Batchsize != integer
#    perm = randperm(Ntrain); # a random permutation of [1:Ntrain] - pick batches as chunks from this array
    for nb=0:Nbatch-1
    perm = randperm(Ntrain); # a random permutation of [1:Ntrain] - pick batches as chunks from this array        # construct batch
        x = [x_train[n] for n in perm[nb*Batchsize+1:(nb+1)*Batchsize]]
        y = [y_train[n] for n in perm[nb*Batchsize+1:(nb+1)*Batchsize]]
        # calculate gradient over the batch
        dw = lossgradient(w,x,y);
        # update weights
        for i=1:length(w)
                w[i] -= lr*dw[i]
        end
    end
    return w
end

In [ ]:
# TRAIN THE NETWORK

In [ ]:
Nepoch = 10000; # For a quick training run
LearningRate = 0.001; # Ditto

In [ ]:
# Collect weights after each epoch in an array (trajectory)
@time weights = [ deepcopy(mytrain!(w,lr=LearningRate)) for epoch=1:Nepoch ];  # copy only copies the top layer, does not descend.

In [ ]:
# Checking if the training worked. Compare the learned function with the actual gaussian
xplot=collect(-Range:0.01:Range) # create an array of x values within the range
y_pred = w[3]'*tanh.(w[1]*xplot' .+ w[2]) .+ w[4] # generate the predicted y values
# plot the converged function, the initial gaussian and the noisy training samples
plot(xplot,[y_pred',exp.(-xplot.^2)]); scatter!(x_train,y_train,leg=false)

In [ ]:
# Plot loss vs epoch
SamplingRate=10;
x = collect(1:SamplingRate:Nepoch);
y = [loss(weights[i],x_train,y_train) for i in x];
plot(x,y)
#plot(x,y,xaxis=:log10,yaxis=:log10) # can also plot in log-log scale

In [ ]:
function diffusiontensor(w,xt,yt,Nb)
    Nweights = length(flat(w)) # number of weights, that is, dimensions of the diffusion tensor
    Nt = length(xt) # number of training examples to be summed over
    prefac = (Nt-Nb)/(2*Nb*(Nt-1))
    V = zeros(Nweights,Nt) # initialize the diffusion matrix
    for i=1:Nt
        x=xt[i]
        y=yt[i]
        V[:,i] = flat(lossgradient(w,[x],[y]))
    end
    V /= Nt;
    dL = flat(lossgradient(w,xt,yt));
    
    return (LearningRate)^2 * prefac * (Nt*V*V' - dL*dL')
end

In [ ]:
ws = flat(weights[1])'; for i=2:Nepoch ws = vcat(ws,flat(weights[i])') end;
wfinalflat = ws[end,:];

In [ ]:
D = diffusiontensor(weights[Nepoch],x_train,y_train,Batchsize)
heatmap(D) # check that it is symmetric, etc.

In [ ]:
# To calculate the Hessian,
# define a function returning elements of the loss-gradient vector dL/dw_j

function lossgradj(w,x,y,j)
    return flat(lossgradient(w,x,y))[j]
end

In [ ]:
# Define a function calculating a column of Hessian matrix:
# Returns d^2(L)/dw_idw_j for all i and given j
lossgradgrad = grad(lossgradj)

In [ ]:
function hessianmatrix(w,x,y)
    Nw = length(flat(w));
    Hess = zeros(Nw,Nw);
    for j=1:Nw
        Hess[:,j] = flat(lossgradgrad(w,x,y,j))
    end
    return Hess
end

In [ ]:
# Using Newton's method get the the true minimum of the full loss function
wf = wfinalflat;
# implement Newton's method to find the true minimum. 4 steps are enough!
for n=1:10
    Hess = hessianmatrix(unflat(wf),x_train,y_train)
    gradwf = flat(lossgradient(unflat(wf),x_train,y_train))
    wf = wf - inv(Hess)*gradwf
end
wminf = wf[:,1]

In [ ]:
Hessmin = hessianmatrix(unflat(wminf),x_train,y_train)

In [ ]:
# diffusiontensor at the minimum
Dmin = diffusiontensor(unflat(wminf),x_train,y_train,Batchsize)

In [ ]:
# This calculation is from Michael's overleaf notes:
# https://www.overleaf.com/2523873322bvvnxpwnskfk
function covariancematrix(D,H;lr=LearningRate)
    F = eigen(H);
    h = F.values
    O = F.vectors
    Nw = length(h)
    ODO = O'*D*O;
    Delta = zeros(Nw,Nw);
    for i=1:Nw
        for j=1:Nw
            Delta[i,j] = ODO[i,j]/(h[i]+h[j])
        end
    end
    return (2/lr)*O*Delta*O'
end
Covmin = covariancematrix(Dmin,Hessmin)

In [ ]:
# test: must be zero
Hessmin*Covmin + Covmin*Hessmin - (2/LearningRate)*Dmin

In [ ]:
# Function that calculates the probability current at a given w
# again, from Michael's notes
function currentvec(w,x,y; lr=LearningRate)
    deltaw = flat(w)-wminf
    H = hessianmatrix(w,x,y);
    return (Dmin*inv(covariancematrix(Dmin,H)) - lr*H)*deltaw 
end

In [ ]:
# Visualizing the vector fields

# scan the vicinity of wstar the get a sense of the current vector field
# Since we want a 2d plot, I scan two weights only. Other weights are fixed to
# their values at wmin.

function gen_grid(wcenter,x_index,y_index,Nx,Ny,Lx,Ly)
    warray = wcenter;
    for nx=1:Nx
        for ny=1:Ny
            dw = wcenter-wcenter # zero
            dw[x_index] = -Lx + 2*Lx*nx/Nx
            dw[y_index] = -Ly + 2*Ly*ny/Ny
            warray = hcat(warray,wcenter + dw)
        end
    end
    return warray
end

x_index = 1;
y_index = 7;

warray = gen_grid(wminf,x_index,y_index,10,10,0.01,0.01)

In [ ]:
# Scatter plot of (w5,w6) values. wstar is shown in red
scatter(warray[x_index,:].-wminf[x_index],warray[y_index,:].-wminf[y_index]); scatter!([0],[0])

In [ ]:
# calculate current vectors
currents = currentvec(unflat(warray[:,1]),x_train,y_train)
npts = length(warray[1,:])
for i=2:npts
    currents = hcat(currents,currentvec(unflat(warray[:,i]),x_train,y_train))
end
currents /= norm(currents[:,2]) # magnitude of current vectors are huge..

In [ ]:
# plot the current vector field
x = warray[x_index,:];
y = warray[y_index,:];
u_cur = currents[x_index,:]/100;
v_cur = currents[y_index,:]/100;
quiver(x.-wminf[x_index], y.-wminf[y_index], quiver=(u_cur, v_cur));  scatter!([0],[0])